# Make tensors for every image

Save the image tensors on disk as `torch.float16`, and the masks as `torch.uint8`

## Imports

In [1]:
# load custom scripts
import config 

# import the necessary packages
from skimage import io
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from imutils import paths

import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import torchvision.transforms.functional as TF
import os

## Load data

In [2]:
# load the image and mask filepaths in a sorted manner
imagePaths = sorted(list(paths.list_images(config.IMAGE_DATASET_PATH)))
maskPaths = sorted(list(paths.list_images(config.MASK_DATASET_PATH)))

# partition the data into training and testing splits using 85% of
# the data for training and the remaining 15% for testing
split = train_test_split(imagePaths, maskPaths, test_size=config.TEST_SPLIT, random_state=42)

# unpack the data split
(trainImages, testImages) = split[:2]
(trainMasks, testMasks) = split[2:]

# write the testing image paths to disk so that we can use then
# when evaluating/testing our model
print("[INFO] saving testing image paths...")
f = open(config.TEST_PATHS, "w")
f.write("\n".join(testImages))
f.close()

[INFO] saving testing image paths...


## Create tensors 

In [3]:
# transforms needed

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
asarray = lambda x: torch.tensor(np.array(x), dtype=torch.uint8)

### Images

In [4]:
# write all created image tensors to disk

# train
startTime = time.time()

for imagePath in trainImages:
    image = io.imread(imagePath)
    image = TF.to_tensor(image.copy())
    image = normalize(image)
    # reduce size so its shorter to load after
    image = torch.as_tensor(image, dtype=torch.float16)
    tensorPath = '/data/jantina/CoralNet/WAPA_RFM/images_tensor_small/train/' + imagePath.split(os.path.sep)[-1].replace("JPG","pt")
    torch.save(image, tensorPath)
    
endTime = time.time()
print("[INFO] total time taken to write the image tensors: {:.2f}s".format(endTime - startTime))

# test
startTime = time.time()

for imagePath in testImages:
    image = io.imread(imagePath)
    image = TF.to_tensor(image.copy())
    image = normalize(image)
    image = torch.as_tensor(image, dtype=torch.float16)
    tensorPath = '/data/jantina/CoralNet/WAPA_RFM/images_tensor_small/test/' + imagePath.split(os.path.sep)[-1].replace("JPG","pt")
    torch.save(image, tensorPath)
    
endTime = time.time()
print("[INFO] total time taken to write the image tensors: {:.2f}s".format(endTime - startTime))


[INFO] total time taken to write the image tensors: 64.31s
[INFO] total time taken to write the image tensors: 11.17s


### Masks

In [5]:
# write all created mask tensors to disk

# train
startTime = time.time()

for maskPath in trainMasks:
    mask = io.imread(maskPath)
    mask = asarray(np.expand_dims(mask, 0))
    tensorPath = '/data/jantina/CoralNet/WAPA_RFM/labels_tensor_small/train/' + maskPath.split(os.path.sep)[-1].replace("png","pt")
    torch.save(mask, tensorPath)
    
endTime = time.time()
print("[INFO] total time taken to write the image tensors: {:.2f}s".format(endTime - startTime))

# test
startTime = time.time()

for maskPath in testMasks:
    mask = io.imread(maskPath)
    mask = asarray(np.expand_dims(mask, 0))
    tensorPath = '/data/jantina/CoralNet/WAPA_RFM/labels_tensor_small/test/' + maskPath.split(os.path.sep)[-1].replace("png","pt")
    torch.save(mask, tensorPath)
    
endTime = time.time()
print("[INFO] total time taken to write the image tensors: {:.2f}s".format(endTime - startTime))

[INFO] total time taken to write the image tensors: 14.47s
[INFO] total time taken to write the image tensors: 2.57s
